In [ ]:
import pandas as pd

df = pd.read_csv("hf://datasets/criteo/FairJob/fairjob.csv.gz")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df.head()

,click,protected_attribute,senior,displayrandom,rank,user_id,impression_id,product_id,cat0,cat1,...,num41,num42,num43,num44,num45,num46,num47,num48,num49,num50
0,0,0,1,1,1,31020,175316,45953,6,6,...,-0.333033,-0.542801,0.074884,-0.751637,-0.299223,-0.647541,-0.252768,-0.577173,-0.264359,-0.737299
1,0,0,1,0,3,515,129699,10569,8,8,...,-0.333033,-0.542801,-0.258659,0.503731,-0.523668,1.088225,0.935563,-0.577173,0.507790,0.610287
2,0,1,1,0,1,12859,134573,51550,6,6,...,-0.333033,-0.542801,-0.236209,-0.111510,0.084159,0.422154,-0.160055,-0.577173,-0.264359,-0.063506
3,0,1,1,0,1,18781,84169,17843,3,3,...,-0.333033,-0.542801,-0.393358,-0.987701,-0.523668,-0.778977,-0.345480,-0.577173,4.368537,-0.996450
4,0,0,1,0,9,19283,76506,11354,8,8,...,-0.333033,-0.542801,1.553377,-0.636441,-0.523668,-0.541088,-0.345480,-0.577173,-0.264359,-0.581808


In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)


X_train = torch.tensor(train_df.drop(columns='click').values, dtype=torch.float32)
y_train = torch.tensor(train_df['click'].values, dtype=torch.long)
X_test = torch.tensor(test_df.drop(columns='click').values, dtype=torch.float32)
y_test = torch.tensor(test_df['click'].values, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
test_dataset  = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=1000)

In [ ]:
len(df.columns)

56

In [ ]:
class Net(nn.Module):
  def __init__(self, input_dim):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(input_dim, 64)
    self.fc2 = nn.Linear(64, 32)
    self.fc3 = nn.Linear(32, 2)
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x.log_softmax(dim=1)

In [ ]:
model = Net(55)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

n_epochs = 1

In [ ]:
def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))


In [ ]:
def test():
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = model(data)
      test_loss += F.nll_loss(output, target, reduction='sum').item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [ ]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()


Test set: Avg. loss: 327956730.5000, Accuracy: 6000/268057 (2%)

Train Epoch: 1 [0/804169 (0%)]	Loss: 1212.464478
Train Epoch: 1 [640/804169 (0%)]	Loss: 27.216831
Train Epoch: 1 [1280/804169 (0%)]	Loss: 0.000000
Train Epoch: 1 [1920/804169 (0%)]	Loss: 0.000000
Train Epoch: 1 [2560/804169 (0%)]	Loss: 176.938782
Train Epoch: 1 [3200/804169 (0%)]	Loss: 0.000000
Train Epoch: 1 [3840/804169 (0%)]	Loss: 0.000000
Train Epoch: 1 [4480/804169 (1%)]	Loss: 0.000000
Train Epoch: 1 [5120/804169 (1%)]	Loss: 53.334812
Train Epoch: 1 [5760/804169 (1%)]	Loss: 0.000000
Train Epoch: 1 [6400/804169 (1%)]	Loss: 69.759262
Train Epoch: 1 [7040/804169 (1%)]	Loss: 0.000000
Train Epoch: 1 [7680/804169 (1%)]	Loss: 21.165966
Train Epoch: 1 [8320/804169 (1%)]	Loss: 0.000000
Train Epoch: 1 [8960/804169 (1%)]	Loss: 0.493055
Train Epoch: 1 [9600/804169 (1%)]	Loss: 0.000726
Train Epoch: 1 [10240/804169 (1%)]	Loss: 0.000000
Train Epoch: 1 [10880/804169 (1%)]	Loss: 0.009419
Train Epoch: 1 [11520/804169 (1%)]	Loss: 0.00